In [1]:
import pandas as pd
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import KeyedVectors
from sklearn.linear_model import Ridge
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import joblib
import nltk
import xgboost
import gc
from xgboost import XGBRegressor
import sklearn
from lightgbm import LGBMRegressor
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import re


In [2]:
mercdata = pd.read_csv("train.tsv", sep='\t')
testData = pd.read_csv("test.tsv", sep = '\t')

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Kalyan
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
lemmatizer = WordNetLemmatizer()

In [23]:
class PreProcess:
    @staticmethod
    def fillBlank(col):
        return col.fillna('')
    
    @staticmethod
    def item_description(desc):
        desc = PreProcess.fillBlank(desc)
        desc = desc.apply(lambda x : x.replace('No description yet', ''))
        return desc
    
    @staticmethod
    def category_null(cat):
        return cat.fillna('//')
    
    @staticmethod
    def fillBrandName(name, brand_name, item_desc):
        if(isinstance(brand_name, float) == False and brand_name != "None"):
             return brand_name
        for i in b_names:
            if(isinstance(i, float)):
                print(i)
                return "WRONG"
            if(i in item_desc or i in name):
                return i
        return "WRONG"

    @staticmethod
    def fillWrongBrandName(c_name, b_name):
        if(b_name == "WRONG"):
            return ''
        return b_name
    
    @staticmethod
    def normalize(col):
        return col / col.max()
    
    @staticmethod
    def str_preprocess(sentence):
        sentence = PreProcess.deEmojify(sentence)
        sentence = sentence.lower()
        sentence = lemmatizer.lemmatize(sentence)
        tokens = tokenizer.tokenize(sentence)
        tokens = list(set(tokens))
        return tokens
    
    @staticmethod
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)
    
    @staticmethod
    def removeStopWords(text):
        stop_words = set(stopwords.words('english'))
        return [item for item in text if item not in stop_words]
    
    @staticmethod
    def fillCategories(t):
        for i in range(len(t)):
            while(len(t[i]) < 3):
                t[i].append("")
        return t

In [6]:
testData.item_description = PreProcess.item_description(testData.item_description)
mercdata.item_description = PreProcess.item_description(mercdata.item_description)

In [7]:
def lenWord(x):
    if(x == ''):
        return 0
    else:
        return len(x.split())

We generate a new column item_description_count which holds the length of each item_description

In [8]:
testData['item_description_count'] = testData.item_description.apply(lenWord)
mercdata['item_description_count'] = mercdata.item_description.apply(lenWord)

In [9]:
testData.item_description_count = PreProcess.normalize(testData.item_description_count)
mercdata.item_description_count = PreProcess.normalize(mercdata.item_description_count)

In [10]:
testData.category_name = PreProcess.category_null(testData.category_name)
mercdata.category_name = PreProcess.category_null(mercdata.category_name)

We generate a list of brand_names to fill null values.

In [11]:
b_names = mercdata[mercdata.brand_name.isna() == False].brand_name.unique()

In [12]:
mercdata.brand_name = mercdata.apply(lambda x: PreProcess.fillBrandName(x['name'], x['brand_name'], x['item_description']), axis=1)
mercdata.brand_name = mercdata.apply(lambda x: PreProcess.fillWrongBrandName(x['category_name'], x['brand_name']), axis=1)

Similarly for Test-data

In [13]:
b_names = testData[testData.brand_name.isna() == False].brand_name.unique()

In [14]:
testData.brand_name = testData.apply(lambda x: PreProcess.fillBrandName(x['name'], x['brand_name'], x['item_description']), axis=1)
testData.brand_name = testData.apply(lambda x: PreProcess.fillWrongBrandName(x['category_name'], x['brand_name']), axis=1)

In [15]:
tokenizer = RegexpTokenizer(r'\w+')

In [16]:
cat = ['name','item_description']

Feature Engineering:

In [17]:
mercdata['name_description'] = mercdata['name'] + mercdata['item_description']
testData['name_description'] = testData['name'] + testData['item_description']
mercdata['name_brand'] = mercdata['name'] + mercdata['brand_name']
testData['name_brand'] = testData['name'] + testData['brand_name']
mercdata['brand_description'] = mercdata['brand_name'] + mercdata['item_description']
testData['brand_description'] = testData['brand_name'] + testData['item_description']

In [18]:
mercdata = mercdata.drop('name', axis = 1)
testData = testData.drop('name', axis = 1)
mercdata = mercdata.drop('item_description', axis = 1)
testData = testData.drop('item_description', axis = 1)

Apply the string pre-processing to the new fields.

In [21]:
l = ['name_description', 'name_brand', 'brand_description']

for i in l:
    mercdata[i] = mercdata[i].apply(PreProcess.str_preprocess)
    testData[i] = testData[i].apply(PreProcess.str_preprocess)

We now remove the stop-words from each of the fields.

In [24]:
l = ['name_description', 'name_brand', 'brand_description']

for i in l:
    mercdata[i] = mercdata[i].apply(PreProcess.removeStopWords)
    testData[i] = testData[i].apply(PreProcess.removeStopWords)

In [25]:
mercdata.category_name = mercdata.category_name.apply(lambda x: lemmatizer.lemmatize(x))
testData.category_name = testData.category_name.apply(lambda x: lemmatizer.lemmatize(x))

In [26]:
t = mercdata.category_name.apply(lambda x : x.split("/", 2))
t = PreProcess.fillCategories(t)

Generate appropriate new columns according to sub-category

In [27]:
mercdata['Category1'] = t.apply(lambda x : x[0])
mercdata['Category2'] = t.apply(lambda x : x[1])
mercdata['Category3'] = t.apply(lambda x : x[2])

Similarly for testData

In [28]:
t = testData.category_name.apply(lambda x : x.split("/", 2))
t = PreProcess.fillCategories(t)
testData['Category1'] = t.apply(lambda x : x[0])
testData['Category2'] = t.apply(lambda x : x[1])
testData['Category3'] = t.apply(lambda x : x[2])

Now that we've split according to sub-categories, we don't need the column category_name anymore.

In [29]:
mercdata = mercdata.drop('category_name', axis = 1)
testData = testData.drop('category_name', axis = 1)

In [30]:
class Embeddings:
    @staticmethod
    def genList(feature, length):
        l = []
        for i in range(1, length + 1):
            l.append(feature + str(i))
        return l
    
    @staticmethod
    def identity_tokenizer(text):
        return text
    
    #returns and concatenates the tf-idf embedding into train and test. Feature_name is the name of the feature to be encoded and num_features is the max number of features which should be outputted.
    @staticmethod
    def tfidf_embed(train, test, feature_name, num_features):
        vectorizer = TfidfVectorizer(tokenizer = Embeddings.identity_tokenizer, lowercase = False, max_features = num_features)
        vectorizer.fit(test[feature_name].append(train[feature_name]))
        nm = vectorizer.transform(test[feature_name])
        df = pd.DataFrame.sparse.from_spmatrix(nm, columns = Embeddings.genList(feature_name, num_features))
        test = pd.concat([df, test], axis = 1)
        nm = vectorizer.transform(train[feature_name])
        df = pd.DataFrame.sparse.from_spmatrix(nm, columns = Embeddings.genList(feature_name, num_features))
        train = pd.concat([df, train], axis = 1)
        return train, test
    
    @staticmethod
    def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
        if len(tokens_list)<1:
            return np.zeros(k)
        if generate_missing:
            vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
        else:
            vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
        length = len(vectorized)
        summed = np.sum(vectorized, axis=0)
        averaged = np.divide(summed, length)
        return averaged

    @staticmethod
    def get_word2vec_embeddings(vectors, feature, generate_missing=False):
        embeddings = feature.apply(lambda x: get_average_word2vec(x, vectors,generate_missing=generate_missing)) 
        return list(embeddings)
    
    @staticmethod
    def labelEncoding(train, test, feature_name):
        le = preprocessing.LabelEncoder()
        le.fit(train[feature_name].append(test[feature_name]))
        train[feature_name] = pd.DataFrame(le.transform(train[feature_name]))
        test[feature_name] = pd.DataFrame(le.transform(test[feature_name]))
        return train, test

Load word2vec. Not used anymore

In [31]:
#file = 'GoogleNews-vectors-negative300.bin.gz'
#word2vec = KeyedVectors.load_word2vec_format(file, binary=True)

We convert the following features into their respective tf-idf embeddings.

In [32]:
l = ['name_description', 'name_brand', 'brand_description']
for i in l:   
    mercdata, testData = Embeddings.tfidf_embed(mercdata, testData, i, 100000)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


For these categorical values we use label-encoding.

In [33]:
l = ['brand_name', 'Category1', 'Category2', 'Category3']
for i in l:
    mercdata, testData = Embeddings.labelEncoding(mercdata, testData, i)

Returns the number of common elements between the lists

In [34]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [35]:
features = intersection(mercdata.columns, testData.columns)

We ensure that both the datasets have common columns.

In [36]:
final_train, final_test = mercdata[features], testData[features]

To remove the skew of the price as shown in the EDA, we apply a log transformation to make it more like a gaussian curve.

In [37]:
fin_price = np.log(1 + mercdata.price)

In [38]:
testID = testData.id

In [39]:
del mercdata, testData

In [40]:
gc.collect()

80

Drop the additional columns, which have already been converted to their respective embeddings.

In [41]:
l = []
for i in final_test.columns:
    if(final_test[i].dtype == 'object'):
        l.append(i)
final_train = final_train.drop(l, axis = 1)
final_test = final_test.drop(l, axis = 1)

We now train our final model on the dataset.

In [42]:
cat = CatBoostRegressor()

In [43]:
cat.fit(final_train, fin_price)

Learning rate set to 0.134359
0:	learn: 0.7343339	total: 25.5s	remaining: 7h 3m 53s
1:	learn: 0.7229059	total: 36.7s	remaining: 5h 5m 7s
2:	learn: 0.7137142	total: 45.9s	remaining: 4h 14m 14s
3:	learn: 0.7065800	total: 54.1s	remaining: 3h 44m 19s
4:	learn: 0.7006062	total: 1m 2s	remaining: 3h 25m 44s
5:	learn: 0.6954823	total: 1m 9s	remaining: 3h 12m 3s
6:	learn: 0.6906156	total: 1m 18s	remaining: 3h 5m 15s
7:	learn: 0.6856159	total: 1m 28s	remaining: 3h 2m 25s
8:	learn: 0.6820326	total: 1m 35s	remaining: 2h 56m 8s
9:	learn: 0.6789530	total: 1m 43s	remaining: 2h 51m 31s
10:	learn: 0.6759054	total: 1m 50s	remaining: 2h 45m 32s
11:	learn: 0.6723493	total: 2m	remaining: 2h 45m 8s
12:	learn: 0.6698327	total: 2m 7s	remaining: 2h 40m 57s
13:	learn: 0.6674517	total: 2m 15s	remaining: 2h 38m 36s
14:	learn: 0.6650015	total: 2m 22s	remaining: 2h 35m 35s
15:	learn: 0.6629989	total: 2m 28s	remaining: 2h 31m 52s
16:	learn: 0.6609547	total: 2m 35s	remaining: 2h 29m 32s
17:	learn: 0.6590586	total: 2m

143:	learn: 0.5796562	total: 15m 46s	remaining: 1h 33m 47s
144:	learn: 0.5791263	total: 15m 53s	remaining: 1h 33m 44s
145:	learn: 0.5788948	total: 15m 59s	remaining: 1h 33m 30s
146:	learn: 0.5785740	total: 16m 5s	remaining: 1h 33m 22s
147:	learn: 0.5783647	total: 16m 10s	remaining: 1h 33m 8s
148:	learn: 0.5779914	total: 16m 17s	remaining: 1h 33m
149:	learn: 0.5777700	total: 16m 23s	remaining: 1h 32m 50s
150:	learn: 0.5774884	total: 16m 30s	remaining: 1h 32m 46s
151:	learn: 0.5772795	total: 16m 35s	remaining: 1h 32m 34s
152:	learn: 0.5770788	total: 16m 40s	remaining: 1h 32m 20s
153:	learn: 0.5768747	total: 16m 46s	remaining: 1h 32m 7s
154:	learn: 0.5766651	total: 16m 51s	remaining: 1h 31m 55s
155:	learn: 0.5764580	total: 16m 57s	remaining: 1h 31m 42s
156:	learn: 0.5762641	total: 17m 2s	remaining: 1h 31m 28s
157:	learn: 0.5760752	total: 17m 7s	remaining: 1h 31m 15s
158:	learn: 0.5758862	total: 17m 12s	remaining: 1h 31m 1s
159:	learn: 0.5756877	total: 17m 17s	remaining: 1h 30m 49s
160:	le

283:	learn: 0.5551280	total: 29m 29s	remaining: 1h 14m 21s
284:	learn: 0.5550213	total: 29m 37s	remaining: 1h 14m 19s
285:	learn: 0.5548975	total: 29m 44s	remaining: 1h 14m 15s
286:	learn: 0.5547468	total: 29m 53s	remaining: 1h 14m 16s
287:	learn: 0.5546475	total: 30m 1s	remaining: 1h 14m 13s
288:	learn: 0.5545508	total: 30m 8s	remaining: 1h 14m 9s
289:	learn: 0.5544539	total: 30m 16s	remaining: 1h 14m 6s
290:	learn: 0.5543576	total: 30m 21s	remaining: 1h 13m 59s
291:	learn: 0.5542585	total: 30m 27s	remaining: 1h 13m 50s
292:	learn: 0.5541607	total: 30m 32s	remaining: 1h 13m 42s
293:	learn: 0.5540669	total: 30m 38s	remaining: 1h 13m 34s
294:	learn: 0.5539164	total: 30m 45s	remaining: 1h 13m 30s
295:	learn: 0.5538251	total: 30m 50s	remaining: 1h 13m 21s
296:	learn: 0.5536782	total: 30m 57s	remaining: 1h 13m 17s
297:	learn: 0.5535706	total: 31m 3s	remaining: 1h 13m 9s
298:	learn: 0.5534201	total: 31m 11s	remaining: 1h 13m 7s
299:	learn: 0.5533256	total: 31m 17s	remaining: 1h 12m 59s
300:

426:	learn: 0.5404240	total: 43m 54s	remaining: 58m 54s
427:	learn: 0.5403422	total: 43m 59s	remaining: 58m 47s
428:	learn: 0.5402714	total: 44m 4s	remaining: 58m 40s
429:	learn: 0.5401904	total: 44m 10s	remaining: 58m 33s
430:	learn: 0.5401267	total: 44m 17s	remaining: 58m 27s
431:	learn: 0.5398477	total: 44m 25s	remaining: 58m 25s
432:	learn: 0.5397793	total: 44m 31s	remaining: 58m 18s
433:	learn: 0.5397117	total: 44m 36s	remaining: 58m 11s
434:	learn: 0.5396329	total: 44m 42s	remaining: 58m 4s
435:	learn: 0.5395668	total: 44m 48s	remaining: 57m 57s
436:	learn: 0.5395025	total: 44m 53s	remaining: 57m 50s
437:	learn: 0.5394153	total: 44m 59s	remaining: 57m 43s
438:	learn: 0.5393520	total: 45m 4s	remaining: 57m 36s
439:	learn: 0.5392888	total: 45m 10s	remaining: 57m 29s
440:	learn: 0.5392193	total: 45m 15s	remaining: 57m 22s
441:	learn: 0.5391111	total: 45m 21s	remaining: 57m 15s
442:	learn: 0.5390512	total: 45m 27s	remaining: 57m 9s
443:	learn: 0.5389847	total: 45m 32s	remaining: 57m 

574:	learn: 0.5297774	total: 58m 29s	remaining: 43m 13s
575:	learn: 0.5297138	total: 58m 35s	remaining: 43m 7s
576:	learn: 0.5296662	total: 58m 40s	remaining: 43m
577:	learn: 0.5296184	total: 58m 45s	remaining: 42m 54s
578:	learn: 0.5295606	total: 58m 51s	remaining: 42m 47s
579:	learn: 0.5294924	total: 58m 58s	remaining: 42m 42s
580:	learn: 0.5294383	total: 59m 3s	remaining: 42m 35s
581:	learn: 0.5293872	total: 59m 9s	remaining: 42m 28s
582:	learn: 0.5293189	total: 59m 14s	remaining: 42m 22s
583:	learn: 0.5292269	total: 59m 20s	remaining: 42m 16s
584:	learn: 0.5291609	total: 59m 26s	remaining: 42m 9s
585:	learn: 0.5290949	total: 59m 32s	remaining: 42m 4s
586:	learn: 0.5290477	total: 59m 38s	remaining: 41m 57s
587:	learn: 0.5289945	total: 59m 43s	remaining: 41m 50s
588:	learn: 0.5289449	total: 59m 48s	remaining: 41m 44s
589:	learn: 0.5288918	total: 59m 54s	remaining: 41m 37s
590:	learn: 0.5288418	total: 59m 59s	remaining: 41m 31s
591:	learn: 0.5287917	total: 1h 5s	remaining: 41m 24s
592

718:	learn: 0.5221148	total: 1h 12m 7s	remaining: 28m 11s
719:	learn: 0.5220777	total: 1h 12m 12s	remaining: 28m 4s
720:	learn: 0.5220403	total: 1h 12m 17s	remaining: 27m 58s
721:	learn: 0.5219894	total: 1h 12m 24s	remaining: 27m 52s
722:	learn: 0.5219507	total: 1h 12m 29s	remaining: 27m 46s
723:	learn: 0.5218979	total: 1h 12m 35s	remaining: 27m 40s
724:	learn: 0.5218544	total: 1h 12m 40s	remaining: 27m 34s
725:	learn: 0.5218008	total: 1h 12m 46s	remaining: 27m 27s
726:	learn: 0.5217606	total: 1h 12m 51s	remaining: 27m 21s
727:	learn: 0.5217197	total: 1h 12m 57s	remaining: 27m 15s
728:	learn: 0.5216834	total: 1h 13m 2s	remaining: 27m 9s
729:	learn: 0.5216461	total: 1h 13m 7s	remaining: 27m 2s
730:	learn: 0.5215867	total: 1h 13m 13s	remaining: 26m 56s
731:	learn: 0.5215487	total: 1h 13m 19s	remaining: 26m 50s
732:	learn: 0.5214986	total: 1h 13m 24s	remaining: 26m 44s
733:	learn: 0.5214493	total: 1h 13m 30s	remaining: 26m 38s
734:	learn: 0.5214122	total: 1h 13m 35s	remaining: 26m 32s
735

858:	learn: 0.5157341	total: 1h 25m 33s	remaining: 14m 2s
859:	learn: 0.5156985	total: 1h 25m 39s	remaining: 13m 56s
860:	learn: 0.5156639	total: 1h 25m 44s	remaining: 13m 50s
861:	learn: 0.5156342	total: 1h 25m 49s	remaining: 13m 44s
862:	learn: 0.5156004	total: 1h 25m 54s	remaining: 13m 38s
863:	learn: 0.5155316	total: 1h 26m 1s	remaining: 13m 32s
864:	learn: 0.5154874	total: 1h 26m 8s	remaining: 13m 26s
865:	learn: 0.5154408	total: 1h 26m 13s	remaining: 13m 20s
866:	learn: 0.5154098	total: 1h 26m 19s	remaining: 13m 14s
867:	learn: 0.5153751	total: 1h 26m 24s	remaining: 13m 8s
868:	learn: 0.5153141	total: 1h 26m 31s	remaining: 13m 2s
869:	learn: 0.5152255	total: 1h 26m 39s	remaining: 12m 56s
870:	learn: 0.5151910	total: 1h 26m 45s	remaining: 12m 50s
871:	learn: 0.5151598	total: 1h 26m 50s	remaining: 12m 44s
872:	learn: 0.5151259	total: 1h 26m 55s	remaining: 12m 38s
873:	learn: 0.5150860	total: 1h 27m	remaining: 12m 32s
874:	learn: 0.5150524	total: 1h 27m 6s	remaining: 12m 26s
875:	le

In [44]:
res = cat.predict(final_test)

We undo the log transformation on our predictions.

In [45]:
res = np.exp(res)
res = res - 1

In [46]:
my_submission4 = pd.DataFrame({'id': testID, 'price': res})

In [47]:
my_submission4.describe()

,id,price
count,4.447610e+05,444761.000000
mean,7.414820e+05,21.926502
std,4.278432e+05,19.223199
min,9.000000e+00,1.466699
25%,3.703390e+05,13.193004
50%,7.418710e+05,17.315884
75%,1.112556e+06,24.508055
max,1.482533e+06,1220.717796


In [ ]:
my_submission4.to_csv('CAT.csv', index=False)